In [2]:
import time
import datetime
import re
import requests
import lxml.html
import numpy as np
import pandas as pd
import email
import smtplib
from email.mime.text import MIMEText #负责构造文本
from email.mime.image import MIMEImage #负责构造图片
from email.mime.multipart import MIMEMultipart #负责连接多个part
from email.header import Header

In [6]:
url = 'https://srh.bankofchina.com/search/whpj/search_cn.jsp'

In [7]:
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Content-Length': '65',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Host': 'srh.bankofchina.com',
    'Origin': 'https://www.boc.cn',
    'Referer': 'https://www.boc.cn/sourcedb/whpj/',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Mobile Safari/537.36'
}


In [21]:
del con

In [26]:
con = pd.DataFrame(columns = ['现汇买入价', '现钞买入价','现汇卖出价', '现钞卖出价', '中行折算价', '发布时间'])

In [29]:
for i in range(1,2187):
    date = datetime.datetime.now().strftime('%Y-%m-%d')
    
    form_data = {
    'erectDate': '2019-04-01',
    'nothing': '2020-04-01',
    'pjname':'英镑',
    'page':'{}'.format(i),
    'head':'head_620.js',
    'bottom':'bottom_591.js'}
    
    page = requests.post(url, headers = headers, data = form_data)
    lxmlobject = lxml.html.fromstring(page.text)
    
    info = lxmlobject.xpath('//div[@class="BOC_main publish"]')[0]
    info1 = info.xpath('string(.)')
    info2 = info1.split('英镑')
    
    for i in info2[1:len(info2)]:
        obs = i.replace('\t','').replace('\r','').split('\n')[1:7]
        newdf = pd.DataFrame(obs).T
        newdf.columns = con.columns
        con = pd.concat([con, newdf], axis = 0, ignore_index = True)
    
    time.sleep(1)

ConnectionError: ('Connection aborted.', OSError(0, 'Error'))

In [6]:
def check_and_send():
    if float(pounds.iloc[len(pounds)-1,2]) < 855:
        mail_host = 'smtp.163.com'
        mail_sender = 'm18008255389@163.com'#发件邮箱
        mail_receiver = 'Xu.Min@warwick.ac.uk'#接收邮箱
        mail_license = 'Minxv520'#邮箱授权码
        mm = MIMEMultipart('related')#
        subject_content = '''买入英镑！'''#邮件主题

        try:
            #邮件头部内容
            mm['From'] = 'Xu Min<m18008255389@163.com>' #注意遵守格式，<>内为发件邮箱
            mm['To'] = 'receiver_1_name<Xu.Min@warwick.ac.uk>' #注意遵守格式，<>内为发件邮箱
            mm['Subject'] = Header(subject_content,'utf-8')

            #添加正文文本
            body_content = '''现在英镑价格为{}，请保持关注'''.format(pounds.iloc[0,2])#邮件正文
            message_text = MIMEText(body_content, 'plain','utf-8')
            mm.attach(message_text) #加入所有文本

            #发送邮件
            stp = smtplib.SMTP(0)#创建SMTP对象，可以为0或('smtp-mail.outlook.com', 25)
            stp.connect(mail_host, 25)#设置发件人邮箱的域名和端口，端口地址为25
            stp.set_debuglevel(1) #固定为1，打印出和SMTP服务器交互的所有信息
            stp.login(mail_sender, mail_license) #登录邮箱：地址和授权码
            stp.sendmail(mail_sender, mail_receiver, mm.as_string()) #发送邮件：发件人和邮件内容
            print('Successfully Sent!')
            stp.quit()
        except:
            print('Something is wrong!')

In [30]:
con

,现汇买入价,现钞买入价,现汇卖出价,现钞卖出价,中行折算价,发布时间
0,877.26,850,883.72,887.63,878.68,2020.04.01 23:38:16
1,877.26,850,883.72,887.63,878.68,2020.04.01 23:38:16
2,877.26,850,883.72,887.63,878.68,2020.04.01 23:38:16
3,877.9,850.62,884.37,888.28,878.68,2020.04.01 23:24:16
4,877.4,850.14,883.86,887.77,878.68,2020.04.01 23:06:25
5,879.46,852.13,885.93,889.85,878.68,2020.04.01 22:54:43
6,879.37,852.04,885.84,889.76,878.68,2020.04.01 22:54:36
7,879.16,851.84,885.63,889.55,878.68,2020.04.01 22:01:16
8,879.3,851.97,885.77,889.69,878.68,2020.04.01 21:53:08
9,878.17,850.88,884.64,888.55,878.68,2020.04.01 21:36:28


In [18]:
for i in range(1,2):
    get_exchange_rates()
    pounds.drop_duplicates(inplace = True)
    check_and_send()
    pounds.to_csv('/Users/apple/Desktop/pounds.csv',header = False, mode = 'a')
    print(time.gmtime())
    #time.sleep(21600)

time.struct_time(tm_year=2020, tm_mon=4, tm_mday=1, tm_hour=17, tm_min=47, tm_sec=34, tm_wday=2, tm_yday=92, tm_isdst=0)


In [20]:
pounds

,现汇买入价,现钞买入价,现汇卖出价,现钞卖出价,中行折算价,发布时间
0,877.26,850,883.72,887.63,878.68,2020.04.01 23:38:16
1,877.9,850.62,884.37,888.28,878.68,2020.04.01 23:24:16
2,877.4,850.14,883.86,887.77,878.68,2020.04.01 23:06:25
3,879.46,852.13,885.93,889.85,878.68,2020.04.01 22:54:43
4,879.37,852.04,885.84,889.76,878.68,2020.04.01 22:54:36
5,879.16,851.84,885.63,889.55,878.68,2020.04.01 22:01:16
6,879.3,851.97,885.77,889.69,878.68,2020.04.01 21:53:08
7,878.17,850.88,884.64,888.55,878.68,2020.04.01 21:36:28
8,877.77,850.5,884.24,888.15,878.68,2020.04.01 21:16:47
9,876.64,849.4,883.09,887,878.68,2020.04.01 20:45:39
